In [2]:
import pandas as pd
import numpy as np

# ex1

In [142]:
company = pd.read_csv('../exe_data/Company.csv')
company_data = pd.read_csv('../exe_data/Company_Data.csv')

In [76]:
# 先对company_data进行处理：
# 1.添加一列为收入所在年份，
# 2.发现有些收入额为负数，暂时选择只保留收入为正的记录
# 3.计算每年每年每个企业各类别的收入占比（按照企业，年份聚合，计算每个类别的收入占比）
# 4. 根据企业、年份聚合，计算对应的熵

In [77]:
company_data['year'] = pd.to_datetime(company_data.日期).dt.year
company_data = company_data[company_data.收入额>0]
company_data['组内收入贡献'] = company_data.groupby(['year','证券代码'])['收入额'].transform(lambda x:x/x.sum())

In [79]:
company_data.head()

,证券代码,日期,收入类型,收入额,year,组内收入贡献
0,1,2008/12/31,1,1.084218e+10,2008,0.146473
1,1,2008/12/31,2,1.259789e+10,2008,0.170192
2,1,2008/12/31,3,1.451312e+10,2008,0.196066
3,1,2008/12/31,4,1.063843e+09,2008,0.014372
4,1,2008/12/31,5,8.513880e+08,2008,0.011502


In [88]:
company_data['组内贡献new'] = company_data.组内收入贡献.apply(lambda x : -x*np.log(x))

In [89]:
company_data.head()

,证券代码,日期,收入类型,收入额,year,组内收入贡献,组内贡献new
0,1,2008/12/31,1,1.084218e+10,2008,0.146473,0.281362
1,1,2008/12/31,2,1.259789e+10,2008,0.170192,0.301381
2,1,2008/12/31,3,1.451312e+10,2008,0.196066,0.319451
3,1,2008/12/31,4,1.063843e+09,2008,0.014372,0.060973
4,1,2008/12/31,5,8.513880e+08,2008,0.011502,0.051359


In [107]:
df_s = company_data.groupby(['证券代码','year'])['组内贡献new'].sum().reset_index()

In [108]:
df_s.head()

,证券代码,year,组内贡献new
0,1,2008,2.068035
1,1,2009,1.671752
2,1,2010,2.108355
3,1,2011,3.137535
4,1,2012,2.698959


In [109]:
df_s.证券代码 = df_s.证券代码.astype('string')
df_s.证券代码 = df_s.证券代码.apply(lambda x: '#'+x.zfill(6))

In [110]:
df_s.year = df_s.year.astype('int')

In [111]:
df_s = df_s.rename(columns = {'year':'日期','组内贡献new':'I'})

In [131]:
df1 = company.merge(df_s,how = 'left', on = ['证券代码','日期'])

In [141]:
df1.head()

,证券代码,日期,I
0,#000007,2014,3.070462
1,#000403,2015,2.790585
2,#000408,2016,2.818541
3,#000408,2017,NaN
4,#000426,2015,3.084266


# EX2

In [111]:
df = pd.read_excel('../exe_data/组队信息汇总表（Pandas）.xlsx')
df2 = df.copy()

In [112]:
# 修改列名，便于使用wide_to_long
df2.columns = ['所在群', '队伍名称', '编号_0', '群昵称_0', '编号_1', '群昵称_1', '编号_2',
       '群昵称_2', '编号_3', '群昵称_3', '编号_4', '群昵称_4', '编号_5',
       '群昵称_5', '编号_6', '群昵称_6', '编号_7', '群昵称_7', '编号_8',
       '群昵称_8', '编号_9', '群昵称_9', '编号_10', '群昵称_10']

In [113]:
# 利用wide_to_long使宽表变长表
df2 = pd.wide_to_long(df2, 
                stubnames = ['群昵称','编号'],
                i = ['所在群', '队伍名称'],
                j = '队内排序', 
                sep = '_', 
                suffix = '.+')

In [114]:
# 去掉空值
df2 = df2.reset_index().dropna()

In [115]:
# 修改index
df2.index = np.arange(df2.shape[0])

In [116]:
# 修改编号的数据类型
df2.编号 = df2.编号.astype('int')

In [117]:
# 添加是否为队长这个字段
df2['是否队长'] = 1
df2['是否队长'] = df2['是否队长'].where(df2.群昵称.isin(df.队长_群昵称), 0)

In [118]:
# 取出有用的字段
df2 = df2[['是否队长', '队伍名称', '群昵称', '编号']]

In [119]:
# 列名修改
df2.rename(columns = {'群昵称':'昵称'}, inplace = True)

In [120]:
df2

,是否队长,队伍名称,昵称,编号
0,1,你说的都对队,山枫叶纷飞,5
1,0,你说的都对队,蔡,6
2,0,你说的都对队,安慕希,7
3,0,你说的都对队,信仰,8
4,0,你说的都对队,biubiu🙈🙈,20
...,...,...,...,...
141,0,七星联盟,Daisy,63
142,0,七星联盟,One Better,131
143,0,七星联盟,rain,112
144,1,应如是,思无邪,54


#  ex3

In [186]:
population = pd.read_csv('../exe_data/county_population.csv')
candidate = pd.read_csv('../exe_data/president_county_candidate.csv')

In [187]:
population.head()
# 需要先将US County 分出两列，县和州

,US County,Population
0,".Autauga County, Alabama",55869
1,".Baldwin County, Alabama",223234
2,".Barbour County, Alabama",24686
3,".Bibb County, Alabama",22394
4,".Blount County, Alabama",57826


In [188]:
candidate.head()

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True


1 有多少县满足总投票数超过县人口数的一半

In [189]:
# candidate 表  
#  按照县聚合，得到每个县投票总人数
#  把state，county 按照population中US County中的格式处理下
#  把每个县投票人数信息添加到county表中,然后找出投票人数超过人口总数的县,并计数

In [190]:
candidate['US County'] = '.'+candidate['county']+', '+candidate['state']
temp = candidate.groupby('US County')['total_votes'].sum().reset_index()
population = population.merge(temp, how = 'left', on = 'US County')

In [191]:
population.head()

,US County,Population,total_votes
0,".Autauga County, Alabama",55869,27770.0
1,".Baldwin County, Alabama",223234,109679.0
2,".Barbour County, Alabama",24686,10518.0
3,".Bibb County, Alabama",22394,9595.0
4,".Blount County, Alabama",57826,27588.0


In [192]:
population[population.total_votes*2>population.Population].shape[0]

1434

结论: 共有1434个县总投票数超过县人数的一半

2 把州（state）作为行索引，把投票候选人作为列名，列名的顺序按照候选人在全美的总票数由高到低排序，行列对应的元素为该候选人在该州获得的总票数

In [207]:
# 先得到全美票数排名
# 然后计算每个候选人在每个州的总票数

In [197]:
candidate.drop('US County',axis = 1, inplace = True)

In [234]:
# 排名
col = candidate.groupby('candidate')['total_votes'].sum().sort_values(ascending = False).index 

# 计算每个候选人在各州所得总票数
temp = pd.DataFrame(candidate.groupby(['state','candidate'])['total_votes'].sum().reset_index()) 

# 透视:长表变宽表
df2 = temp.pivot(index = 'state', columns = 'candidate', values = 'total_votes')[col].fillna(0).astype('int')

In [242]:
df2.head()

candidate,Joe Biden,Donald Trump,Jo Jorgensen,Howie Hawkins,Write-ins,Rocky De La Fuente,Gloria La Riva,Kanye West,Don Blankenship,Brock Pierce,...,Tom Hoefling,Ricki Sue King,Princess Jacob-Fambro,Blake Huber,Richard Duncan,Joseph Kishore,Jordan Scott,Gary Swing,Keith McCormic,Zachary Scalf
state,,,,,,,,,,,,,,,,,,,,,
Alabama,849648,1441168,25176,0,7312,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alaska,153405,189892,8896,0,34210,318,0,0,1127,825,...,0,0,0,0,0,0,0,0,0,0
Arizona,1672143,1661686,51465,0,2032,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Arkansas,423932,760647,13133,2980,0,1321,1336,4099,2108,2141,...,0,0,0,0,0,0,0,0,0,0
California,11109764,6005961,187885,81025,80,60155,51036,0,0,0,...,0,0,0,0,0,0,0,0,0,0


3  每一个州下设若干县，定义拜登在该县的得票率减去川普在该县的得票率为该县的BT指标，若某个州所有县BT指标的中位数大于0，则称该州为Biden State，请找出所有的Biden State

In [243]:
# 先计算每个县里面,每个候选人的得票率
# 计算每个县BT指标
# 根据BT指标计算每个州的中位数
# 筛选出所有中位数大于0的州

In [284]:
candidate.head()

,state,county,candidate,party,total_votes,won,rate
0,Delaware,Kent County,Joe Biden,DEM,44552,True,0.511945
1,Delaware,Kent County,Donald Trump,REP,41009,False,0.471232
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False,0.011997
3,Delaware,Kent County,Howie Hawkins,GRN,420,False,0.004826
4,Delaware,New Castle County,Joe Biden,DEM,195034,True,0.678065


In [285]:
# 计算每个县的得票率
candidate['rate'] = candidate.groupby(['state', 'county'])['total_votes'].transform(lambda x: x/x.sum())

In [286]:
temp = candidate.copy()

In [287]:
# 只保留川普和拜登的投票率数据
temp = temp[temp.candidate.isin(['Donald Trump', 'Joe Biden'])]

In [288]:
# 长表变宽表,把拜登和川普作为列名
temp = temp.pivot(index = ['state', 'county'], columns = 'candidate', values = 'rate').reset_index()

In [289]:
# 计算每个县的BT指数
temp['BT'] = temp['Joe Biden'] - temp['Donald Trump']

In [290]:
# 计算每个州BT指数的中位数
final = temp.groupby(['state'])['BT'].median().reset_index()

In [293]:
# 筛选出BT指数大于0的州
final[final.BT>0].state

4               California
6              Connecticut
7                 Delaware
8     District of Columbia
11                  Hawaii
21           Massachusetts
30              New Jersey
39            Rhode Island
45                 Vermont
Name: state, dtype: object